In [1]:
import pandas as pd
import datetime as dt
from binance.client import Client
# install python-binance
# pip install python-binance
import time
import numpy
import requests
import sqlite3
import threading

In [2]:

## connecting to the sqlite database

con = sqlite3.connect('quant.db', check_same_thread=False)  
cur = con.cursor()


In [3]:
## check database exist

def tables_in_sqlite_db(con):
    cur = con.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [
        v[0] for v in cur.fetchall()
        if v[0] != "sqlite_sequence"
    ]
    cur.close()
    return tables



print(tables_in_sqlite_db(con))

['transactions']


In [4]:
def database_sqlite_update(buy_or_sell, timestamp, symbol, quantity, price,transaction_cost):
    global con   
    global cur 
    
    sqlite_command = 'INSERT INTO transactions(buy_or_sell, timestamp, symbol, quantity, price,transaction_cost) VALUES'+"('"+buy_or_sell+"','"+str(timestamp)+"','"+symbol+"',"+str(quantity) +","+ str(price) +","+str(transaction_cost)+")"      
    
    print(sqlite_command)
    
    cur.execute(sqlite_command)
    con.commit()
    
    

database_sqlite_update('BUY','1498793709153','BTCUSDT',100, 30000,35.14)
database_sqlite_update('SELL','1498793709153','BTCUSDT',50, 30000,35.14)

INSERT INTO transactions(buy_or_sell, timestamp, symbol, quantity, price,transaction_cost) VALUES('BUY','1498793709153','BTCUSDT',100,30000,35.14)
INSERT INTO transactions(buy_or_sell, timestamp, symbol, quantity, price,transaction_cost) VALUES('SELL','1498793709153','BTCUSDT',50,30000,35.14)


In [5]:

def get_transaction_history():

    global con 
    global cur 


    df = pd.read_sql_query("SELECT * from transactions", con)

    # Verify that result of SQL query is stored in the dataframe

    return df
get_transaction_history()

,trans_no,buy_or_sell,timestamp,symbol,quantity,price,transaction_cost
0,6,BUY,1498793709153,BTCUSDT,100.0,30000.0,35.14
1,7,BUY,1498793709153,BTCUSDT,100.0,30000.0,35.14
2,8,BUY,1498793709153,BTCUSDT,100.0,30000.0,35.14
3,9,BUY,1498793709153,BTCUSDT,100.0,30000.0,35.14
4,10,SELL,1498793709153,BTCUSDT,50.0,30000.0,35.14
5,11,BUY,1498793709153,BTCUSDT,100.0,30000.0,35.14
6,12,SELL,1498793709153,BTCUSDT,50.0,30000.0,35.14
7,13,BUY,1498793709153,BTCUSDT,100.0,30000.0,35.14
8,14,SELL,1498793709153,BTCUSDT,50.0,30000.0,35.14
9,15,BUY,1498793709153,BTCUSDT,100.0,30000.0,35.14


In [6]:
## get your position

In [7]:
def get_position():
    
    #df = pd.read_sql_query("SELECT SUM(quantity) as buy_total from transactions WHERE buy_or_sell = 'BUY' ", con)
    #buy_total = get_position()['buy_total'][0]
    #df = pd.read_sql_query("SELECT SUM(quantity) as sell_total from transactions WHERE buy_or_sell = 'SELL' ", con)
    #sell_total = get_position()['sell_total'][0]
    
    buy_total = pd.read_sql_query("SELECT SUM(quantity) as buy_total from transactions WHERE buy_or_sell = 'BUY' ", con)['buy_total'][0]
    sell_total = pd.read_sql_query("SELECT SUM(quantity) as sell_total from transactions WHERE buy_or_sell = 'SELL' ", con)['sell_total'][0]
    
    
    return buy_total - sell_total
    
    #return buy_total-sell_total

# Alert: this function is just for developers

In [8]:

def delete_all_transaction():
    cur.execute("DELETE FROM transactions ;")
    cur.execute("UPDATE transactions SET trans_no = 0 ;")
    con.commit()

# Defining the function for calculating Bollinger Bands

In [9]:
# define bolinger band
#1m 3m 5m 15m 30m 1h 2h 4h 6h 8h 12h 1d 3d 1w 1M
def bollingerband(symbol, width, intervalunit, length):

    if intervalunit == '1T':# 依需求設定
        start_str = '100 minutes ago UTC'
        interval_data = '1m' 

        D = pd.DataFrame(client.get_historical_klines(symbol=symbol, start_str=start_str, interval=interval_data))
        D.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_trades',
                         'taker_base_vol', 'taker_quote_vol', 'is_best_match']
        D['open_date_time'] = [dt.datetime.fromtimestamp(x / 1000) for x in D.open_time]
        
        
        D['symbol'] = symbol
        D = D[['symbol', 'open_date_time', 'open', 'high', 'low', 'close', 'volume', 'num_trades', 'taker_base_vol',
                   'taker_quote_vol']]
    
    if intervalunit == '1D':# 依需求設定
        start_str = '50 day ago UTC'
        # "1 Jan, 2017"
        
        interval_data = '1d' 

        D = pd.DataFrame(client.get_historical_klines(symbol=symbol, start_str=start_str, interval=interval_data))
        D.columns = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'qav', 'num_trades',
                         'taker_base_vol', 'taker_quote_vol', 'is_best_match']
        D['open_date_time'] = [dt.datetime.fromtimestamp(x / 1000) for x in D.open_time]
        
        #print(D)
        
        D['symbol'] = symbol
        D = D[['symbol', 'open_date_time', 'open', 'high', 'low', 'close', 'volume', 'num_trades', 'taker_base_vol',
                   'taker_quote_vol']]
    
    
    
    

    df = D.set_index("open_date_time")

    df['close'] = df['close'].astype(float)

    df = df['close']

    df1 = df.resample(intervalunit).agg({"close": "last"})

    unit = width
    
    std = numpy.std(df1['close'][len(df1) - length:len(df1)])

    band1 = unit * std

    bb_center = numpy.mean(df1['close'][len(df1) - length:len(df1)])

    band_high = bb_center + band1

    band_low = bb_center - band1

    return band_high, bb_center, band_low, std



# defining the function for running the bot and stopping the bot

In [23]:
run_flag = 1
def run_robot():

    #exceute bot
    
    
    while run_flag==1:
    
        _interval_unit = '1D'
        # _interval_unit = '1'
        
        bb_1m = bollingerband(symbo1_trade, width, _interval_unit, length)
    
        if _interval_unit == '1T' : ##one minute 
        
            print('1 minute upper center lower: ', bb_1m)
            
        elif _interval_unit == '1D':
            
            print('1 day upper center lower and standard deviation: ', bb_1m)
    
        marketprice = 'https://api.binance.com/api/v1/ticker/24hr?symbol=' + symbo1_trade
        res = requests.get(marketprice)
        data = res.json()
        lastprice = float(data['lastPrice'])
    
        print(lastprice)
    
        
    
        try:
            if lastprice > bb_1m[0]:
                print('sell')
                
                # make market order on binance
                #client.order_market_sell(symbol=symbo1_trade, quantity=orderquantity)
                
                break
                #the loop stops if the order is made
        except:
            pass
    
        try:
            if lastprice < bb_1m[2]:
                print('buy')
                
                # make market order on binance
                #client.order_market_buy(symbol=symbo1_trade, quantity=orderquantity)
                
                break
                #the loop stops if the order is made        
        except:
            pass
    
        time.sleep(1)

    print('finish running')

In [10]:
def stop_robot():
    global run_flag
    run_flag = 0
    time.sleep(5)
    run_flag = 1


# Setting paremeter for our robot

In [11]:
with open('api_key_secret.txt') as f:
    lines = f.read().splitlines() 
    f.close()
    

# API Key (You need to get these from Binance account)
api_key = lines[0]
api_secret = lines[1]

client = Client(api_key=api_key, api_secret=api_secret)

# ticker of product
symbo1_trade = 'BTCUSDT'
#'BTCBUSD'
#'XRPBUSD'

# order quantity (more than 10 USDT)
orderquantity = 35

# bollingerband length and width
length = 20 #lookback period
width = 2 # standard deviation

# Run the robot using thread

t = threading.Thread(target = run_robot)
t.start()

# Stop the robot

stop_robot()

# Trading strategy (shoud change the lastprice to  (bid and ask))

In [24]:
run_flag = 1
def bot_1():
    global con
    global cur
    
    while run_flag==1:
    
        _interval_unit = '1D'
        # _interval_unit = '1T'

        upperband, mean , lowerband ,  std_real= bollingerband(symbo1_trade, width, '1D', length)
        mean_plus_1_std = mean + std_real
        mean_minus_1_std = mean - std_real

        marketprice = 'https://api.binance.com/api/v1/ticker/24hr?symbol=' + symbo1_trade
        res = requests.get(marketprice)
        data = res.json()
        lastprice = float(data['lastPrice'])
        
        entryZ=2
        exitZ=1
        
        
        z_score_last_price = (lastprice-mean)/std_real
        
        quantity = 100
        
        ## should change lastprice to (bid and ask)
        
        pos = get_position()
        
        
        print(z_score_last_price)
        
        
        try:
           
            if pos==0 and (z_score_last_price< -entryZ):
                
                # make market order on binance
                #client.order_market_buy(symbol=symbo1_trade, quantity=orderquantity)
                
                #database_sqlite_update (Buy_or_Sell, timestamp, Ticker, quantity, price, transaction cost)
                database_sqlite_update('BUY','1498793709153','BTCUSDT',quantity, lastprice, quantity*0.005*lastprice)
                
                
            elif pos> 0 and (z_score_last_price > entryZ):
                
                # make market order on binance
                #client.order_market_buy(symbol=symbo1_trade, quantity=orderquantity)
                
                #database_sqlite_update (Buy_or_Sell, timestamp, Ticker, quantity, price, transaction cost)
                database_sqlite_update('SELL','1498793709153','BTCUSDT',2*quantity, lastprice, quantity*0.005*lastprice)
            
            elif pos> 0 and (z_score_last_price > exitZ):
                
                # make market order on binance
                #client.order_market_buy(symbol=symbo1_trade, quantity=orderquantity)
                
                #database_sqlite_update (Buy_or_Sell, timestamp, Ticker, quantity, price, transaction cost)
                database_sqlite_update('SELL','1498793709153','BTCUSDT',quantity, lastprice, quantity*0.005*lastprice)
                
                
        
        except:
            pass
        
        
        
        
        
    print('finish running')
        
        
        
'''
        try:
            if lastprice > bb_1m[0]:
                print('sell')
                
                # make market order on binance
                #client.order_market_sell(symbol=symbo1_trade, quantity=orderquantity)
                
                break
                #the loop stops if the order is made
        except:
            pass
    
        try:
            if lastprice < bb_1m[2]:
                print('buy')
                
                # make market order on binance
                #client.order_market_buy(symbol=symbo1_trade, quantity=orderquantity)
                
                break
                #the loop stops if the order is made        
        except:
            pass
    
        time.sleep(1)
'''

    
    

"\n        try:\n            if lastprice > bb_1m[0]:\n                print('sell')\n                \n                # make market order on binance\n                #client.order_market_sell(symbol=symbo1_trade, quantity=orderquantity)\n                \n                break\n                #the loop stops if the order is made\n        except:\n            pass\n    \n        try:\n            if lastprice < bb_1m[2]:\n                print('buy')\n                \n                # make market order on binance\n                #client.order_market_buy(symbol=symbo1_trade, quantity=orderquantity)\n                \n                break\n                #the loop stops if the order is made        \n        except:\n            pass\n    \n        time.sleep(1)\n"

if (pos==0 & zscoreAsk < -entryZ ) % entry only id=placeOrder(symbol,secType,exchange, '', 0, '', '',...
localSymbol, 'BUY', numContract, 'MKT',0,0,'DAY',''); pos=numContract;
elseif (pos < 0 & zscoreAsk < -entryZ) % exit and entry id=placeOrder(symbol,secType,exchange, '', 0, '', '',...
localSymbol, 'BUY', 2*numContract, 'MKT',0,0,'DAY',''); pos=numContract;
elseif (pos < 0 & zscoreAsk < -exitZ) % exit only id=placeOrder(symbol,secType,exchange, '', 0, '', '',...
localSymbol, 'BUY', numContract, 'MKT',0,0,'DAY',''); pos=0;
elseif (pos==0 & zscoreBid > entryZ) % entry only id=placeOrder(symbol,secType,exchange, '', 0, '', '',...
localSymbol, 'SELL', numContract, 'MKT',0,0,'DAY',''); pos=-numContract;
elseif (pos > 0 & zscoreBid > entryZ ) % exit and entry id=placeOrder(symbol,secType,exchange, '', 0, '', '',...
localSymbol, 'SELL', 2*numContract, 'MKT',0,0,'DAY',''); pos=-numContract;
elseif (pos > 0 & zscoreBid > exitZ ) % exit only id=placeOrder(symbol,secType,exchange, '', 0, '', '',...
localSymbol, 'SELL', numContract, 'MKT',0,0,'DAY',''); pos=0;
end

# Run Robot

In [25]:
t = threading.Thread(target = bot_1)
t.start()

-0.3023854522046521
-0.3023854522046521
-0.30275737049271384
-0.30275737049271384
-0.30331158527165175
-0.30331545942071375
-0.30331158527165175
-0.3032813394565252
-0.3032813394565252
-0.31312159547588564
-0.3100571692903277
-0.3083387370616493
-0.3072269468789842
-0.3128595006266974
-0.3089531648724735
-0.3089531648724735
-0.30689231109728426
-0.3070167401172709
-0.3062458298929077
-0.30628456910016166
-0.3062917727228534
-0.30629564665139547
-0.30712854128776956
-0.30342893953085326
-0.3061344887618704
-0.3061306146642262
-0.30599535551439694
-0.30599535551439694
-0.30672365762150977
-0.3073047497282495
-0.307321999734039
-0.307321999734039
-0.30674479319569126
-0.3080531582750174
-0.30693361155533155
-0.30673991835123365
-0.30730613846324645
-0.30673668034151136


# Stop the robot

In [26]:
stop_robot()

-0.30674055420628304
finish running
